<a href="https://colab.research.google.com/github/issmythe/CS194App/blob/master/gcvi_forecast_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
#@title Installations
!pip install fsspec kaleido gcsfs geopandas &> /dev/null

In [2]:
#@title Authentications
from google.colab import auth, drive

auth.authenticate_user()
drive.mount('/content/drive')

# ! gcloud auth login
# ! gcloud auth application-default login

Mounted at /content/drive


In [3]:
#@title Imports
import itertools
import os
import sys
import time

# import geopandas as gpd
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

# Plotting
import plotly
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots

# Analysis
from scipy.stats import pearsonr

import sklearn
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import cross_val_predict, HalvingGridSearchCV

# Helpers
CODE_PATH = 'drive/MyDrive/code/in_season/'
sys.path.append(CODE_PATH)
sys.path.append('content/' + CODE_PATH)

from data_utils import *
from model_utils import *


In [4]:
#@title Constants and misc.
get_rmse = lambda x, y: mean_squared_error(x, y, squared=False)

pd.options.mode.chained_assignment = None
FIG_PATH = 'drive/MyDrive/figs/in_season/%s/' % datetime.today().strftime('%Y%m')
! mkdir -p $FIG_PATH

# Data and linear model helpers

##**Setup

In [5]:
GEN_DATA = False

In [6]:
#@title Read in data
if GEN_DATA:
    gcvi_obs = read_vi_ts(
        'gs://nsf-phase2/vi_timeseries/s2_%d_timeseries_sr_corrected_noclouds.csv', 'GCVI')
else:
    gcvi_obs = pd.read_csv(
        'gs://nsf-phase2/vi_timeseries/kenya_all_years/sample_years_no_clouds.csv')
    gcvi_obs = gpd.GeoDataFrame(
        gcvi_obs, geometry=gpd.points_from_xy(gcvi_obs['x'], gcvi_obs['y']))\
        .drop(['x', 'y'], axis=1)
    gcvi_obs['ts'] = pd.to_datetime(gcvi_obs['ts'])

vi_max_df = pd.read_csv(
    'gs://nsf-phase2/in_season/precomputed_features/partial_season_vis_filtered.csv')

pred_df_full = pd.read_csv(
    'gs://nsf-phase2/in_season/precomputed_features/full_season_filtered.csv')
ids = pred_df_full[['id']]

def get_id_from_tuple(x):
    lon, lat, year, _ = x
    data = [int(year), round(lat, 6), round(lon, 6)]
    return '_'.join([str(x) for x in data])

gcvi_obs['id'] = gcvi_obs.apply(get_vi_id, axis=1)
gcvi_obs['id'] = gcvi_obs['id'].apply(get_id_from_tuple)
gcvi_obs = gcvi_obs[['id', 'year', 'ts', 'GCVI']].rename({'GCVI': 'gcvi'}, axis=1).drop_duplicates()


In [7]:
#@title Get daily GCVI df, last observed value filled forward

def vis_all_dates_ffill(df):
    df['t'] = \
        df['ts'].dt.dayofyear - df['year'].apply(lambda x: datetime(x, 3, 1)).dt.dayofyear

    dates = pd.DataFrame({'t': [x for x in range(df['t'].min(), df['t'].max() + 1)]})
    vi_all = ids.merge(dates, how='cross').merge(df, how='left')
    vi_all = vi_all.sort_values('t')
    vi_all['gcvi'] = vi_all.groupby('id')['gcvi'].fillna(method='ffill')
    return vi_all, dates

# Get column with dates of most recent Sentinel reading for field
def add_last_obs_date(df):
    df['last_obs'] = df['ts']
    df.loc[~df['last_obs'].isna(), 'last_obs'] = \
        df.loc[~df['last_obs'].isna(), 't']
    df['last_obs'] = df.groupby('id')['last_obs'].fillna(method='ffill')
    df['last_obs'] = (df['last_obs'] - df['last_obs'].min()).fillna(0)
    return df

def add_cumulative_max(df, end_t=None):
    if end_t is None:
        start_date = datetime(2022, 3, 1)
        end_t = (datetime(2022, 9, 1) - start_date).days

    # Filter for growing season and get cumulative maximum
    df = df[(df['t'] >= 0) & (df['t'] < end_t)]
    df['gcvi_curr_season'] = df['gcvi']
    df.loc[df['last_obs'] < 59, 'gcvi_curr_season'] = None
    df['cum_max'] = df.groupby('id')['gcvi'].cummax()
    df['cum_max_season'] = df.groupby('id')['gcvi_curr_season'].cummax().fillna(df['cum_max'])

    # Get mean GCVI/GCVI cumulative max by day of year
    daily_avg = df.groupby('t')['gcvi'].mean().reset_index()\
                    .rename({'gcvi': 'gcvi_daily_avg'}, axis=1)\
                    .sort_values('t')
    daily_avg['cum_max_daily_avg'] = daily_avg['gcvi_daily_avg'].cummax()
    df = df.merge(daily_avg)

    # Fill in missing values with daily maximums
    df['gcvi'] = df['gcvi'].fillna(df['gcvi_daily_avg'])
    df['cum_max'] = df['cum_max'].fillna(df['cum_max_daily_avg'])
    df['cum_max_season'] = df['cum_max_season'].fillna(df['cum_max_daily_avg'])
    return df

def get_forward_interp_vi_df(obs_df, end_t=None):
    all_dates, dates = vis_all_dates_ffill(obs_df)
    all_dates = add_last_obs_date(all_dates)
    cum_max_df = add_cumulative_max(all_dates, end_t)

    # Add true max GCVI (dependent var) and clean up year columns
    cum_max_df = cum_max_df.merge(vi_max_df[['id', 'gcvi_max_full']])
    cum_max_df['year'] = cum_max_df['id'].apply(lambda x: int(x.split('_')[0]))

    return cum_max_df.drop_duplicates(), dates

gcvi_all, dates = get_forward_interp_vi_df(gcvi_obs)

In [8]:
#@title Read in Fourier time series
harmonic_ts = pd.read_csv('gs://nsf-phase2/harmonics_10iters/GCVI_harmonics.csv')
harmonic_ts = harmonic_ts[['DATE', 'GCVI_HARMFIT', 'id']]\
    .rename({'DATE': 'date', 'GCVI_HARMFIT': 'gcvi'}, axis=1)

harmonic_ts['date'] = pd.to_datetime(harmonic_ts['date'])
harmonic_ts['year'] = harmonic_ts['date'].dt.year
harmonic_ts['doy'] = harmonic_ts['date'].dt.dayofyear
harmonic_ts = harmonic_ts.sort_values('date')

hist_id_df = harmonic_ts[['id']].drop_duplicates()
hist_id_df['cleaned_id'] = hist_id_df['id'].apply(get_id_tuple).apply(get_id_from_tuple)
harmonic_ts = harmonic_ts.merge(hist_id_df).drop('id', axis=1).rename({'cleaned_id': 'id'}, axis=1)
harmonic_ts = harmonic_ts.merge(vi_max_df[['id', 'gcvi_max_full']])


## ** Helpers

In [9]:
#@title Get plot labels
start_date = datetime(2022, 3, 1)
end_t = (datetime(2022, 9, 1) - start_date).days

dates['date'] = dates['t'].apply(lambda x: start_date + timedelta(x))
dates = dates[(dates['t'] >= 0) & (dates['t'] < end_t)]
labels = dates.assign(day=dates['date'].dt.day, month=dates['date'].dt.month)
labels = labels[labels['day'] == 15]
labels['label'] = labels.apply(lambda x: f"{x['month']}/{x['day']}", axis=1)


In [10]:
#@title Helper to get cross-validated predictions from linear regression
def quick_predict(df, pred_cols, use_window=True):
    def _predict_group(g):
        g = g.reset_index(drop=True)
        group_kfold = GroupKFold(n_splits=4).split(g, groups=g['year'])
        group_preds = cross_val_predict(
            linear_model.LinearRegression(), g[pred_cols], g['gcvi_max_full'], cv=group_kfold)
        return group_preds, g['gcvi_max_full']

    if use_window:
        df = df[df['t'] < end_t]
    preds = df.groupby('t').apply(_predict_group).reset_index()
    preds['r2'] = preds[0].apply(lambda x: pearsonr(x[0], x[1])[0] ** 2)
    preds['rmse'] = preds[0].apply(lambda x: get_rmse(x[0], x[1]))
    return preds.drop(0, axis=1)

def get_historical_preds(df, pred_col, filter_test=False):
    if filter_test:
        df = df[df['year'] == df['obs_year']]
    hist_r2 = pearsonr(df[pred_col], df['gcvi_max_full'])[0] ** 2
    hist_rmse = ((df[pred_col] - df['gcvi_max_full']) ** 2).mean() ** 0.5
    return baseline_preds[['t']].assign(r2=hist_r2, rmse=hist_rmse)

def format_output_reduced_form(test_preds, expected_df=None):
    expected_df = expected_df if expected_df is not None else gcvi_all.merge(hist_max)
    pred_df = expected_df.merge(pd.concat(test_preds)[['id', 't', 'pred']], how='left')
    pred_df['cont_pred'] = \
        pred_df.groupby(['id'])['pred'].fillna(method='ffill').fillna(pred_df['hist_max'])

    quick_r2 = lambda g: pearsonr(g['cont_pred'], g['gcvi_max_full'])[0] ** 2
    quick_rmse = lambda g: get_rmse(g['cont_pred'], g['gcvi_max_full'])
    r2_df = pred_df.groupby('t').apply(quick_r2).reset_index().rename({0: 'r2'}, axis=1)
    rmse_df = pred_df.groupby('t').apply(quick_rmse).reset_index().rename({0: 'rmse'}, axis=1)
    return r2_df.merge(rmse_df)

def add_n_since_max(df):
    df = df.assign(lt_max=0, group=0)
    df.loc[df['gcvi'] < df['cum_max'], 'lt_max'] = 1
    df.loc[np.abs(df['gcvi'] - df['cum_max']) < 1e-6, 'group'] = 1

    df['group'] = df.groupby('id')['group'].cumsum()
    df['n_since_max'] = df.groupby(['id', 'group'])['lt_max'].cumsum()
    return df

def predict_rolling_window(pred_df, cols, window_size=5, expected_df=None,
                           filter_test=False, get_coef=False):
    obs_df = pred_df.dropna()
    test_preds, coefficients = [], []

    for year in range(2016, 2020):
        for t in range(end_t):
            train = obs_df[(obs_df['year'] != year) & (np.abs(obs_df['t'] - t) <= window_size)]
            test = obs_df[(obs_df['year'] == year) & (obs_df['t'] == t)]
            if filter_test:
                test = test[test['year'] == test['obs_year']]

            if len(test) > 0:
                lm = linear_model.LinearRegression().fit(train[cols], train['gcvi_max_full'])
                test_preds.append(test.assign(pred=lm.predict(test[cols])))
                coefficients.append({'year': year, 't': t,
                                     **{col + '_coef': coef for col, coef in zip(cols, lm.coef_)}})
    if get_coef:
        return format_output_reduced_form(test_preds, expected_df), pd.DataFrame(coefficients)
        # return pd.concat(test_preds), pd.DataFrame(coefficients)
    return format_output_reduced_form(test_preds, expected_df)


In [11]:
#@title Confidence interval code
def bootstrap_ci(df, pred_fn, n=100, verbose=True, fname=None):
    if fname:
        try:
            return pd.read_csv(
                f'drive/MyDrive/data/feature_forecasting/confidence_intervals/{fname}.csv')
        except FileNotFoundError:
            pass

    np.random.seed(123)
    t0 = time.time()
    preds = []
    for i in range(n):
        sample_ids = ids.sample(frac=1, replace=True)
        sample_df = df.merge(sample_ids)
        preds.append(pred_fn(sample_df).assign(iter=i))
        if verbose and (i + 1) % 10 == 0:
            print(f'{i + 1}/{n} folds complete; {round(time.time() - t0, 1)} elapsed.')

    all_preds = pd.concat(preds)
    lower_ci = all_preds.groupby('t')[['r2', 'rmse']].quantile(0.025).reset_index()\
        .rename({'r2': 'r2_lower', 'rmse': 'rmse_lower'}, axis=1)
    upper_ci = all_preds.groupby('t')[['r2', 'rmse']].quantile(1 - 0.025).reset_index()\
        .rename({'r2': 'r2_upper', 'rmse': 'rmse_upper'}, axis=1)
    ci_df = lower_ci.merge(upper_ci)
    if fname:
        ci_df.to_csv(
            f'drive/MyDrive/data/feature_forecasting/confidence_intervals/{fname}.csv', index=False)
    return ci_df

def quick_predict_historical(df, pred_col):
    hist_r2 = pearsonr(df[pred_col], df['gcvi_max_full'])[0] ** 2
    hist_rmse = ((df[pred_col] - df['gcvi_max_full']) ** 2).mean() ** 0.5
    return pd.DataFrame(dict(t=[t for t in range(end_t)])).assign(r2=hist_r2, rmse=hist_rmse)

In [12]:
#@title Updated plotting helpers
from matplotlib import colors

def quick_plot_results(plot_objects, fname=None, show=True):
    fig = plotly.subplots.make_subplots(rows=1, cols=2,
            subplot_titles=['R2, Predicted vs. Peak GCVI', 'RMSE, Predicted vs. Peak GCVI'])

    for po in plot_objects:
        po.plot(fig)

    rmse_max = max([po.preds['rmse'].max() for po in plot_objects])
    fig.update_xaxes(title='Date', tickvals=labels['t'], ticktext=labels['label'])
    fig.update_yaxes(range=[0, 1], row=1, col=1)
    fig.update_yaxes(range=[0, rmse_max * 1.02], row=1, col=2)

    fig.update_layout(height=500, width=1000)
    if fname:
        plotly.io.write_image(fig, f'{FIG_PATH}/{fname}.png', scale=2)
    if show:
        fig.show()

class PlotObject(object):
    def __init__(self, preds, name, kwargs, cis=None):
        super(object, self).__init__()
        self.preds = preds
        self.name = name
        self.kwargs = kwargs
        self.cis = cis

    def get_ci_rgba_str(self, kwargs):
        r, g, b, _ = colors.to_rgba(kwargs['line_color'])
        return f'rgba({r},{g},{b},0.2)'

    def add_prediction_to_plot(self, fig, df, name, kwargs, row, cis=None):
        fig.add_trace(
            go.Scatter(name=name, x=df['t'], y=df['r2'], **kwargs), row=row, col=1)
        fig.add_trace(
            go.Scatter(showlegend=False, x=df['t'], y=df['rmse'], **kwargs), row=row, col=2)

        def _add_ci(cis, var, col):
            if cis is None:
                return

            fig.add_trace(go.Scatter(
                    x=list(cis['t']) + list(cis['t'][::-1]),
                    y=list(cis[f'{var}_upper']) + list(cis[f'{var}_lower'][::-1]),
                    fill='toself',
                    fillcolor=self.get_ci_rgba_str(kwargs),
                    line=dict(color='rgba(255,255,255,0)'),
                    showlegend=False), row=row, col=col)

        _add_ci(cis, 'r2', 1)
        _add_ci(cis, 'rmse', 2)

    def plot(self, fig, row=1):
        self.add_prediction_to_plot(fig, self.preds, self.name, self.kwargs, row=row, cis=self.cis)



## ** Get baseline preds and historical data

In [13]:
#@title Get df of historical VI curves
nharmonics, omega = 2, 1

if GEN_DATA:
    # https://colab.research.google.com/drive/1l4LcW0_eD8wjFnv6k-Q2FWy1daXcFnFR?authuser=2#scrollTo=iVxNsOppm_W6
    hist_harms = pd.concat([
        pd.read_csv(f'gs://nsf-phase2/harmonics_1iter/kenya/{y}.csv').assign(im_year=y)
        for y in range(2016, 2023)]) # TODO
    hist_harms['year'] = hist_harms['id'].apply(lambda x: int(x.split('_')[0]))

    def get_id(row):
        return '_'.join([str(row['year']),
                         str(round(row['PTLAT'], 6)),
                         str(round(row['PTLON'], 6))])

    def fourier(t, *coeffs):
        vi_fit = coeffs[0] + coeffs[1] * t
        for n in range(nharmonics):
            timerad = t * 2 * (n + 1) * np.pi * omega
            vi_fit += coeffs[2 + n*2] * np.cos(timerad) + coeffs[3 + n*2] * np.sin(timerad)
        return vi_fit

    def get_fourier(row):
        order = ['fourier_c', 'fourier_t', 'fourier_cos1',
                'fourier_sin1', 'fourier_cos2', 'fourier_sin2']
        params = [row[var] for var in order]
        return lambda t: fourier(t, *params)

    start = (datetime(2017, 3, 1) - datetime(2017, 1, 1)).days
    end = (datetime(2017, 11, 1) - datetime(2017, 1, 1)).days

    def get_vi_ts(row):
        fourier_fn = get_fourier(row)
        return [fourier_fn(i / 365) for i in range(end - start)]

    hist_harms = hist_harms[hist_harms['id'].isin(ids['id'])]
    hist_harms['year_id'] = hist_harms['id'] + '_' + hist_harms['im_year'].apply(str)
    hist_harms = hist_harms.drop_duplicates(
        subset=[x for x in hist_harms.columns if x != 'system:index'])
    hist_harms['vi_ts'] = hist_harms.apply(get_vi_ts, axis=1)
    hist_harms = hist_harms[['year_id', 'vi_ts']].explode('vi_ts').assign(
        doy=[x for x in range(start, end)] * hist_harms['year_id'].nunique())

    hist_harms['year'] = hist_harms['year_id'].apply(lambda x: int(x.split('_')[0]))
    hist_harms['im_year'] = hist_harms['year_id'].apply(lambda x: int(x.split('_')[-1]))

else:
    hist_harms = pd.read_csv('gs://nsf-phase2/harmonics_1iter/kenya/all_years_by_day.csv')

hist_id_to_id = hist_harms[['year_id']].drop_duplicates()
hist_id_to_id['id'] = hist_id_to_id['year_id'].apply(lambda x: x[:-5])
hist_harms = hist_harms.merge(hist_id_to_id)

hist_harms['vi_orig'] = hist_harms['vi_ts']
hist_harms.loc[hist_harms['vi_ts'] < gcvi_all['gcvi'].min(), 'vi_ts'] = gcvi_all['gcvi'].min()
hist_harms.loc[hist_harms['vi_ts'] > gcvi_all['gcvi'].max(), 'vi_ts'] = gcvi_all['gcvi'].max()

hist_harms_med = hist_harms[hist_harms['year'] != hist_harms['im_year']]\
    .groupby(['id', 'doy'])['vi_ts'].median().reset_index().rename({'vi_ts': 'vi_ts_med'}, axis=1)
hist_harms_mean = hist_harms[hist_harms['year'] != hist_harms['im_year']]\
    .groupby(['id', 'doy'])['vi_ts'].mean().reset_index().rename({'vi_ts': 'vi_ts_mean'}, axis=1)
hist_harms_avg = hist_harms_med.merge(hist_harms_mean)
hist_harms_avg['vi_ts'] = hist_harms_avg['vi_ts_med']

In [14]:
#@title Get historical OOS maximum
# Constrain to OOS years
oos_max = hist_harms[hist_harms['year'] != hist_harms['im_year']]

# Get in-season maximum
max_start = (datetime(2023, 4, 1) - datetime(2023, 1, 1)).days
max_end = (datetime(2023, 9, 1) - datetime(2023, 1, 1)).days
hist_max = oos_max[(oos_max['doy'] >= max_start) & (oos_max['doy'] < max_end)]
hist_max = hist_max.merge(hist_max.groupby('year_id')['vi_ts'].max().reset_index())

# Remove unrealistic values
lower, upper = pred_df_full['gcvi_max'].min(), pred_df_full['gcvi_max'].max()
hist_max = hist_max[(hist_max['vi_ts'] >= lower) & (hist_max['vi_ts'] <= upper)]

# Get mean of peaks and fill na with OOS average
fill_na_max = pd.DataFrame({'year': [y for y in range(2016, 2020)]}).assign(
    oos_mean_max=[hist_max.loc[hist_max['im_year'] != y, 'vi_ts'].mean()
                  for y in range(2016, 2020)])
hist_max['id'] = hist_max['year_id'].apply(lambda x: x[:-5])
hist_max = hist_max.groupby('id')['vi_ts'].mean().reset_index()
hist_max = hist_max.merge(
    harmonic_ts[['id', 'year', 'gcvi_max_full']].drop_duplicates(), how='right')
hist_max = hist_max.merge(fill_na_max)
hist_max['vi_ts'] = hist_max['vi_ts'].fillna(hist_max['oos_mean_max'])
hist_max = hist_max.rename({'vi_ts': 'hist_max'}, axis=1)
print(round(pearsonr(hist_max['gcvi_max_full'], hist_max['hist_max'])[0] ** 2, 3))
gcvi_all = gcvi_all.merge(hist_max)


0.583


In [15]:
#@title Get baseline and combo predictions for comparison
baseline_preds = quick_predict(gcvi_all, ['cum_max'])
combo_preds = quick_predict(gcvi_all, ['cum_max', 'hist_max'])


##**Get all years data

In [16]:
#@title Read in GCVI all years
gcvi_all_years = pd.read_csv(
    'gs://nsf-phase2/vi_timeseries/kenya_all_years/processed/all_years.csv').merge(
    gcvi_all[['id', 'year']].drop_duplicates().rename({'year': 'obs_year'}, axis=1))
gcvi_obs_ay = gcvi_all_years.dropna(subset='ts')
gcvi_max_ay = gcvi_obs_ay.groupby(['id', 'year'])['gcvi'].max().reset_index().merge(gcvi_obs_ay)
gcvi_max_ay['year_delta'] = np.abs(gcvi_max_ay['year'] - gcvi_max_ay['obs_year'])


<ipython-input-16-b17b9a823191>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  gcvi_all_years = pd.read_csv(


In [17]:
#@title Make prediction dataframe for all years
sent_max = gcvi_all_years.dropna(subset='ts')\
    .groupby(['id', 'year', 'obs_year'])['gcvi'].max().reset_index()\
    .rename({'year': 'im_year'}, axis=1)
sent_max = sent_max.merge(sent_max.groupby(['id'])['gcvi'].mean()
    .reset_index().rename({'gcvi': 'mean_max'}, axis=1)).merge(
    sent_max.groupby(['id'])['im_year'].nunique()
    .reset_index().rename({'im_year': 'n_obs'}, axis=1))
sent_max['hist_max'] = (
    sent_max['mean_max'] * sent_max['n_obs'] - sent_max['gcvi']) / (sent_max['n_obs'] - 1)

all_years_pred_df = gcvi_all_years.rename({'gcvi_max_full': 'gcvi_max_ins'}, axis=1)\
    .merge(sent_max.drop(['mean_max', 'n_obs'], axis=1)\
    .rename({'im_year': 'year', 'gcvi': 'gcvi_max_full'}, axis=1))

# Use harmonic fit for in-sample maxima
idx = (all_years_pred_df['year'] == all_years_pred_df['obs_year'])
all_years_pred_df.loc[idx, 'gcvi_max_full'] = all_years_pred_df.loc[idx, 'gcvi_max_ins']

max_gcvi = gcvi_all['gcvi_max_full'].max() * 1.1
all_years_pred_df = all_years_pred_df[(all_years_pred_df['gcvi_max_full'] > 0) &
                                      (all_years_pred_df['hist_max'] > 0) &
                                      (all_years_pred_df['gcvi_max_full'] < max_gcvi) &
                                      (all_years_pred_df['hist_max'] < max_gcvi)].drop_duplicates()


In [18]:
del gcvi_all_years, max_gcvi, idx, sent_max, gcvi_obs_ay, gcvi_max_ay
import gc
gc.collect()

152

# Workspace

In [19]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [20]:
pred_t = 60
pred_df = gcvi_all[gcvi_all['t'] <= pred_t].groupby(['id', 'year'])['gcvi'].apply(list)\
    .reset_index().merge(gcvi_all[['id', 'gcvi_max_full']].drop_duplicates())

pred_df = all_years_pred_df[all_years_pred_df['t'] <= pred_t]\
    .groupby(['id', 'year', 'obs_year'])['gcvi'].apply(list)\
    .reset_index().merge(gcvi_all[['id', 'gcvi_max_full']].drop_duplicates())


In [21]:
#@title Train/test + model data helpers
def train(model, device, train_loader, optimizer, criterion):
    running_loss = 0.0

    model.train()
    for batch_idx, (x, target) in enumerate(train_loader):
        inputs, labels = x.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return (running_loss / (batch_idx + 1)) ** 0.5


def test(model, device, test_x, test_y, criterion, predict=False):
    model.eval()
    with torch.no_grad():
        data = torch.Tensor(test_x).to(device)
        target = torch.Tensor(test_y).to(device)
        outputs = model(data)
        detached = outputs.detach().cpu().numpy().flatten() if device.type == 'cuda' \
            else outputs.detach().numpy().flatten()
        if predict:
            return detached
        # print('*', outputs.shape, target.shape)
        mse = criterion(outputs, target).item()
        corr = pearsonr(detached, test_y.flatten())[0]

    return mse ** 0.5, corr


class ModelData:
    def __init__(self, pred_df, year, val_year=None, iter=0):
        self.ss_x = StandardScaler()
        self.ss_y = StandardScaler()
        self.train_pd, self.val_pd, self.test_pd = \
            self.get_train_val_test(pred_df, year, iter, val_year)
        self.train_x, self.train_y, self.train_ds, self.train_loader = \
            self.get_model_data(self.train_pd, ['gcvi'], True)
        self.test_x, self.test_y, self.test_ds, self.test_loader = \
            self.get_model_data(self.test_pd, ['gcvi'], False)
        self.val_x, self.val_y, self.val_ds, self.val_loader = \
            self.get_model_data(self.val_pd, ['gcvi'], False)


    def get_train_val_test(self, pred_df, year, iter=0, val_year=None):
        train_pd = pred_df[pred_df['year'] != year].sample(frac=1, random_state=123)
        test_pd = pred_df[pred_df['year'] == year]
        if val_year is None:
            start = int(iter * 0.2 * len(train_pd))
            end = int((iter + 1) * 0.2 * len(train_pd))
            return (train_pd.iloc[np.r_[0: start, end: len(train_pd)]],
                    train_pd.iloc[start: end],
                    test_pd)

        return (train_pd[train_pd['year'] != val_year],
                pred_df[pred_df['year'] == val_year],
                test_pd)

    def get_model_data(self, df, ts_cols, train, batch_size=64):
        x = np.stack([np.array(df[c].to_list()) for c in ts_cols], axis=1)
        n, vars, days = x.shape
        x = x.reshape(n, 1, vars, days)

        y = np.array(df['gcvi_max_full']).reshape(n, 1)
        y = self.ss_y.fit_transform(y) if train else self.ss_y.transform(y)

        ds = TensorDataset(torch.Tensor(x).type(torch.float32),
                           torch.Tensor(y).type(torch.float32))
        loader = DataLoader(ds, batch_size=batch_size)
        return x, y, ds, loader

In [23]:
#@title Make net
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):
    def __conv_calc(self, in_dim, pad, stride, k):
        out = int(np.floor((in_dim + 2 * pad - (k - 1) - 1) / stride + 1))
        return out

    def __init__(self, x_dim, cdim1=8, ldim=8, dropout=0.1, filter_sz=7, stride=4, verbose=False):
        super(Net, self).__init__()
        self.verbose = verbose

        self.conv2d = nn.Conv2d(1, cdim1, (1, filter_sz), stride)
        c1_dim = self.__conv_calc(x_dim, 0, stride, filter_sz)

        self.dropout1 = nn.Dropout2d(dropout)
        flattened_dim = int(c1_dim / 2) * cdim1

        self.cnn_fc = nn.Linear(flattened_dim, ldim)
        self.fc = nn.Linear(ldim, 1)

    def quick_print(self, x, i):
        if self.verbose:
            print(i, x.shape)
        return i + 1

    def forward(self, x):
        i = self.quick_print(x, 0)

        # Convolution #1
        x = self.conv2d(x)
        x = F.relu(x)
        x = self.dropout1(x)
        i = self.quick_print(x, i)  # 1

        x = torch.flatten(x, 2)
        i = self.quick_print(x, i)  # 2

        # Average pool
        x = F.max_pool1d(x, 2)
        i = self.quick_print(x, i)  # 3
        x = torch.flatten(x, 1)
        i = self.quick_print(x, i)  # 4

        # CNN FC layer
        x = self.cnn_fc(x)
        x = F.relu(x)
        i = self.quick_print(x, i)  # 5

        return self.fc(x)  # .flatten()



ss_x = StandardScaler()
ss_y = StandardScaler()

dummy_md = ModelData(pred_df, 2016, val_year=2017)
x, _, _, _ = dummy_md.get_model_data(pred_df[:5], ['gcvi'], train=True)

my_nn = Net(x.shape[-1], cdim1=2, ldim=16, dropout=0.1, verbose=True)
optimizer = optim.SGD(my_nn.parameters(), lr=0.001)
optimizer.zero_grad()

test_im = torch.from_numpy(x[:2]).type(torch.float32)
result = my_nn(test_im)
result

0 torch.Size([2, 1, 1, 61])
1 torch.Size([2, 2, 1, 14])
2 torch.Size([2, 2, 14])
3 torch.Size([2, 2, 7])
4 torch.Size([2, 14])
5 torch.Size([2, 16])


tensor([[0.2074],
        [0.2054]], grad_fn=<AddmmBackward0>)

In [24]:
#@title Prediction helpers
class TrainObject:
    def __init__(self, model_args):

        t0 = time.time()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.criterion = nn.MSELoss()
        dummy_md = ModelData(pred_df, 2016, val_year=2017)
        x, _, _, _ = dummy_md.get_model_data(pred_df, ['gcvi'], train=True)

        self.model = Net(x.shape[3], cdim1=model_args['cdim'], ldim=model_args['ldim'],
                         dropout=model_args['dropout'], filter_sz=model_args['filter_sz'],
                         stride=model_args['stride'], verbose=False)
        self.model.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=model_args['lr'])

        self.epochs = model_args['epochs']

def do_epoch(md, to, min_rmse, epoch, t0, verbose):
    train_rmse = train(to.model, to.device, md.train_loader, to.optimizer, to.criterion)
    val_rmse, val_corr = test(to.model, to.device, md.val_x, md.val_y, to.criterion)
    if verbose:
        test_rmse, test_corr = test(to.model, to.device, md.test_x, md.test_y, to.criterion)
        print(f'{epoch + 1} / {to.epochs} complete. Train: {round(train_rmse, 3)}.',
            f'Val: {round(val_rmse, 3)} {round(val_corr, 3)}',
            f'Test: {round(test_rmse, 3)} {round(test_corr, 3)}',
            round(time.time() - t0, 2))
    return min(min_rmse, val_rmse), val_rmse, val_corr

def quick_predict_year(
    year, model_args, iter=0, val_year=None, tol=None, verbose=True, return_objects=False):
    # Get data and set up model
    md = ModelData(pred_df, year, iter=iter, val_year=val_year)
    to = TrainObject(model_args)

    # Initialize rmse and correlation
    val_rmse, val_corr = test(to.model, to.device, md.val_x, md.val_y, to.criterion)
    rmse, corr, min_rmse, n_inc = [val_rmse], [val_corr], val_rmse, 0
    if verbose:
        print(f'{0} / {max_epochs} complete.', f'Test: {round(val_rmse, 3)} {round(val_corr, 3)}')

    # Train
    t0 = time.time()
    for epoch in range(to.epochs):
        min_rmse, val_rmse, val_corr = do_epoch(md, to, min_rmse, epoch, t0, verbose)
        rmse, corr = rmse + [val_rmse], corr + [val_corr]

        n_inc = n_inc + 1 if val_rmse > min_rmse else 0
        if tol is not None and n_inc >= tol:
            break

    # Predict
    if return_objects:
        return md, to

    preds = test(to.model, to.device, md.test_x, md.test_y, to.criterion, predict=True)
    return to.model, md.test_pd[['year', 'obs_year', 'id', 'gcvi_max_full']].assign(
        pred=md.ss_y.inverse_transform(preds.reshape(md.test_y.shape)).flatten())

In [69]:
#@title Grid search set-up
def get_model_data_rmse(to, md, to_predict):
    if to_predict == 'train':
        x, y, df = md.train_x, md.train_y, md.train_pd
    if to_predict == 'val':
        x, y, df = md.val_x, md.val_y, md.val_pd
    if to_predict == 'test':
        x, y, df = md.test_x, md.test_y, md.test_pd

    preds = test(to.model, to.device, x, y, to.criterion, predict=True)
    preds = df[['year', 'obs_year', 'id', 'gcvi_max_full']].assign(
            pred=md.ss_y.inverse_transform(preds.reshape(y.shape)).flatten())
    preds_ins = preds[preds['year'] == preds['obs_year']]
    return (get_rmse(preds['gcvi_max_full'], preds['pred']),
            get_rmse(preds_ins['gcvi_max_full'], preds_ins['pred']))

param_grid = {'cdim': [2, 4, 8, 16],
              'ldim': [2, 4, 8, 16],
              'dropout': [0, 0.1, 0.2],
              'lr': [5e-4, 1e-3, 5e-3],
              'filter_sz': [3, 7, 10],
              'stride': [2, 4, 6],
              'epochs': [25, 50, 100]}

keys, values = zip(*param_grid.items())
param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

N_COMBOS = 1
np.random.seed(123)
param_combinations = np.random.choice(
    param_combinations, len(param_combinations), replace=False)[:N_COMBOS]


In [ ]:
#@title Do grid search
i = 0
t0 = time.time()

for test_year in range(2016, 2020):
    for val_year in range(test_year + 1, 2020):
        results = []
        for p in range(len(param_combinations)):
            model_params = param_combinations[p]
            md, to = quick_predict_year(
                test_year, model_params, val_year=val_year, verbose=False, return_objects=True)

            row = model_params.copy()
            row['test_year'], row['val_year'], row['p'] = test_year, val_year, p
            row['train_rmse_all'], row['train_rmse_ins'] = get_model_data_rmse(to, md, 'train')
            row['val_rmse_all'], row['val_rmse_ins'] = get_model_data_rmse(to, md, 'val')
            row['test_rmse_all'], row['test_rmse_ins'] = get_model_data_rmse(to, md, 'test')
            results.append(row)

            i += 1
            print(f'{test_year}, {val_year}, {i} / {N_COMBOS} complete',
                  f'{round((time.time() - t0) / 60, 1)} minutes elapsed')

        pd.DataFrame(results).to_csv(
            f'gs://nsf-phase2/in_season/deep_learning/{test_year}_{val_year}.csv', index=False)


In [106]:
#@title Process grid search results
dir = 'gs://nsf-phase2/in_season/deep_learning'
results = pd.concat([
    pd.read_csv(f'{dir}/{y1}_{y2}.csv') for y1 in range(2016, 2020) for y2 in range(y1 + 1, 2020)])

counts = pred_df_full.groupby(['year'])['id'].count().reset_index().rename({'id': 'n'}, axis=1)
param_cols = ['cdim', 'ldim', 'dropout', 'lr', 'filter_sz', 'stride', 'epochs']
rename_dict = {'test_rmse_ins': 'val_rmse_ins', 'test_year': 'val_year', 'val_year': 'test_year'}
results = pd.concat([results[param_cols + ['test_year', 'val_year', 'val_rmse_ins']],
    results[param_cols + ['test_year', 'val_year', 'test_rmse_ins']].rename(rename_dict, axis=1)])\
    .merge(counts.rename({'year': 'val_year'}, axis=1))

results['mse_w'] = (results['val_rmse_ins'] ** 2) * results['n']
grouped_results = results.groupby(param_cols + ['test_year'])[['n', 'mse_w']].sum().reset_index()
grouped_results['rmse'] = (grouped_results['mse_w'] / grouped_results['n']) ** 0.5

In [ ]:
#@title Predict with train/val split by year
tol, max_epochs = 1, 10

obs_years = pred_df[['id', 'obs_year']].drop_duplicates()
preds = []

for test_year in range(2016, 2020):
    for val_year in range(2016, 2022):
    # for val_year in range(5):
        if test_year == val_year:
            continue
        # model, iter_preds = quick_predict_year(test_year, val_year=val_year)
        model, iter_preds = quick_predict_year(test_year, val_year=val_year, tol=tol, max_epochs=max_epochs)
        iter_preds = iter_preds.merge(obs_years[obs_years['obs_year'] == test_year])
        preds.append(iter_preds.assign(val_year=val_year))

dl_preds = pd.concat(preds).groupby(['year', 'id']).mean().reset_index()


In [ ]:
dl_preds

In [ ]:
df = dl_preds#[dl_preds['year'] != 2019]
r2 = pearsonr(df['gcvi_max_full'], df['pred'])[0] ** 2
rmse = ((df['gcvi_max_full'] - df['pred']) ** 2).mean() ** 0.5
# (0.4326105359863152, 1.374710508448396)
r2, rmse

(0.4091454791476172, 1.4085011226107258)

In [ ]:
year = 2017
train_pd, val_pd, test_pd = get_train_val_test(year)
train_x, train_y, train_ds, train_loader = get_model_data(train_pd, ss_x, ss_y, ['gcvi'], True)
test_x, test_y, test_ds, test_loader = get_model_data(test_pd, ss_x, ss_y, ['gcvi'], False)
val_x, val_y, val_ds, val_loader = get_model_data(val_pd, ss_x, ss_y, ['gcvi'], False)

dfx, dfy, dfpd = train_x, train_y, train_pd
dfx, dfy, dfpd = val_x, val_y, val_pd
dfx, dfy, dfpd = test_x, test_y, test_pd

plot_preds = test(model, device, dfx, dfy, criterion, predict=True)
plot_preds = dfpd[['year', 'id', 'gcvi_max_full']].assign(
    pred=ss_y.inverse_transform(plot_preds.reshape(dfy.shape)).flatten())

print(round(pearsonr(plot_preds['gcvi_max_full'], plot_preds['pred'])[0] ** 2, 3),
      round(((plot_preds['gcvi_max_full'] - plot_preds['pred']) ** 2).mean() ** 0.5, 3))
print(baseline_preds[baseline_preds['t'] == pred_t])

# sns.scatterplot(x=preds['gcvi_max_full'], y=preds['pred'])
sns.scatterplot(x=plot_preds['gcvi_max_full'], y=plot_preds['pred'])

In [ ]:
print(baseline_preds[baseline_preds['t'] == pred_t])
sns.scatterplot(x=baseline_preds['t'], y=baseline_preds['r2'])

# Scratch

In [ ]:
#@title Grid search test run
# cdim, ldim, dropout, lr, filter_sz, filter_stride, epochs

def get_model_data_rmse(to, md, to_predict):
    if to_predict == 'train':
        x, y, df = md.train_x, md.train_y, md.train_pd
    if to_predict == 'val':
        x, y, df = md.val_x, md.val_y, md.val_pd
    if to_predict == 'test':
        x, y, df = md.test_x, md.test_y, md.test_pd

    preds = test(to.model, to.device, x, y, to.criterion, predict=True)
    preds = df[['year', 'obs_year', 'id', 'gcvi_max_full']].assign(
            pred=md.ss_y.inverse_transform(preds.reshape(y.shape)).flatten())
    preds_ins = preds[preds['year'] == preds['obs_year']]
    return (get_rmse(preds['gcvi_max_full'], preds['pred']),
            get_rmse(preds_ins['gcvi_max_full'], preds_ins['pred']))

param_grid = {'cdim': [2, 4, 8, 16],
              'ldim': [2, 4, 8, 16],
              'dropout': [0, 0.1, 0.2],
              'lr': [5e-4, 1e-3, 5e-3],
              'filter_sz': [3, 7, 10],
              'stride': [2, 4, 6],
              'epochs': [25, 50, 100]}

keys, values = zip(*param_grid.items())
param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
np.random.seed(123)
param_combinations = np.random.choice(param_combinations, 100, replace=False)

test_year, val_year = 2016, 2017
results = []

i, t0 = 0, time.time()
for model_params in param_combinations:
    row = model_params

    # try:
    md, to = quick_predict_year(
        test_year, model_params, val_year=val_year, verbose=False, return_objects=True)
    row['train_rmse_all'], row['train_rmse_ins'] = get_model_data_rmse(to, md, 'train')
    row['val_rmse_all'], row['val_rmse_ins'] = get_model_data_rmse(to, md, 'val')
    row['test_rmse_all'], row['test_rmse_ins'] = get_model_data_rmse(to, md, 'test')
    # except:
    #     print('!!!!! Params failed', model_params)

    results.append(row)

    if (i + 1) % 10 == 0:
        print('Recording results so far...')
        pd.DataFrame(results).to_csv(
            'gs://nsf-phase2/in_season/deep_learning/initial_param_performance.csv', index=False)

    i += 1
    print(f'{i} / 100 complete, {round(time.time() - t0)} elapsed')

